# PandasDataFrameOutputParser
- 사용자로부터 임의의 DataFrame을 받으면, model이 해당 DataFrame에서 사용자 질문에 따라 데이터를 추출하고 사전 형태로 데이터를 조회할 수 있도록 해준다.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [26]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문서화 목적으로만 사용됩니다.
import pprint
from typing import Any, Dict


def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():  # 파서 출력의 키들을 순회합니다.
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [5]:
import pandas as pd
df = pd.read_csv("./data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [36]:
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import PromptTemplate


parser = PandasDataFrameOutputParser(dataframe=df)
df_query = "Retrieve the first row."
# print(parser.get_format_instructions())
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# chain = prompt | model | parser
chain = prompt | model | parser

def extract_with_llm(query: str):
    output = chain.invoke({"query": query})
    return output
    # return format_parser_output(output)


In [37]:
extract_with_llm(df_query)

{'0': PassengerId                          1
 Survived                             0
 Pclass                               3
 Name           Braund, Mr. Owen Harris
 Sex                               male
 Age                               22.0
 SibSp                                1
 Parch                                0
 Ticket                       A/5 21171
 Fare                              7.25
 Cabin                              NaN
 Embarked                             S
 Name: 0, dtype: object}

In [38]:
extract_with_llm("Retrieve the average of the ages from row 0 to 4.")

OutputParserException: Request 'Here is the formatted query:

mean:Age[0..4]' is not correctly formatted.                     Please refer to the format instructions.

In [39]:
extract_with_llm("Retrieve the average Fare.")

OutputParserException: Request 'Here is the formatted query:

mean:Fare' is not correctly formatted.                     Please refer to the format instructions.